### TO observe RLC, for now, use uncontainzerized version

This script setup the experiments, assuming that the l4s part has been installed correctly. 

In [1]:
import subprocess

tx0_prefix = "ssh PeterYao@pc490.emulab.net"
router0_prefix = "ssh PeterYao@pc500.emulab.net"
router1_prefix = "ssh PeterYao@pc487.emulab.net"
rx0_prefix = "ssh PeterYao@pc816.emulab.net"

nodes_prefix = [tx0_prefix, router0_prefix, router1_prefix, rx0_prefix]

class node:
    def __init__(self, node_ssh_prefix) -> None:
        self.ssh_prefix = node_ssh_prefix

    def execute(self, command, background=False):
        if background:
            print("executing in background")
            # full_command = f"{self.ssh_prefix} 'setsid nohup {command} > /dev/null 2>&1 &'"
            full_command = f'{self.ssh_prefix} "{command}"'
            subprocess.Popen(full_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        else:
            full_command = f'{self.ssh_prefix} "{command}"'
            result = subprocess.run(full_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            if result.returncode == 0:
                print(result.stdout.decode('utf-8'))
            else:
                print(f"Error: {result.stderr.decode('utf-8')}")
        return None
        
tx0_node = node(tx0_prefix)
delay_node = node(router0_prefix)
router_node = node(router1_prefix)
rx0_node = node(rx0_prefix)

nodes = [tx0_node, delay_node, router_node, rx0_node]





In [2]:
from fabric import Connection


tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)




delay = Connection(
    host='pc500.emulab.net',
    user = 'PeterYao',
    port=22,
)


router = Connection(
    host='pc487.emulab.net',
    user='PeterYao',
    port=22,
)

rx = Connection(
    host='pc816.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

conns = [router, delay, tx, rx]


# Experiment 1 50-50 base thp and latency

In [ ]:
commands_noecn = "bash -c 'sudo sysctl -w net.ipv4.tcp_congestion_control=cubic; sudo sysctl -w net.ipv4.tcp_ecn=0'"
for node in nodes:
    node.execute(commands_noecn)
    
print("validating...")
for node in nodes:
    node.execute("sudo sysctl net.ipv4.tcp_congestion_control")
    node.execute("sudo sysctl net.ipv4.tcp_ecn")

In [ ]:
# for some reason, this cannot be automated

for node in nodes:
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

for node in nodes:
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git
cd iproute2
sudo chmod +x configure
sudo ./configure
sudo make
sudo make install"""

router.run(cmd_dualpi2)
router.run("sudo modprobe sch_dualpi2")

In [ ]:
packages = ['iperf3', 'net-tools', 'moreutils']
for conn in conns:
    for package in packages:
        conn.sudo(f'sudo apt update; apt-get -y install {package}')

In [ ]:
offloads = ["gro", "lro", "gso", "tso"]

for conn in conns[:3]:
    for offload in offloads:
        conn.sudo(f'ethtool -K eno3 {offload} off', warn=True)
        conn.sudo(f'ethtool -K enp5s0f0 {offload} off', warn=True)
        conn.sudo(f'ethtool -K enp4s0f0 {offload} off',     warn=True)
        conn.sudo(f'ethtool -K enp6s0f3 {offload} off',    warn=True)
        
        
        
        
        



In [ ]:
rx.sudo('ethtool -K vlan171 gro off')
rx.sudo('ethtool -K vlan171 lro off')
rx.sudo('ethtool -K vlan171 gso off')
rx.sudo('ethtool -K vlan171 tso off')

In [15]:
# modify the delay on the delay node
base_rtt = 25
delay_interfaces = ["enp5s0f0", "eno3"]

for e in delay_interfaces:
    cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=base_rtt/2)
    delay.run(cmds)

In [ ]:
delay.run("sudo tc qdisc show dev enp5s0f0")
delay.run("sudo tc qdisc show dev eno3")

In [1]:
# set up the btl node
n_bdp = 2
base_rtt = 25
btl_capacity = 100 #in Mbps

# fixed values
btl_limit    = int(1000*n_bdp*btl_capacity*base_rtt/8) # limit of the bottleneck, n_bdp x BDP in bytes 
packet_number=int(btl_limit/1500)+1

print("btl limit: ", btl_limit)
print("packet number: ", packet_number)

btl limit:  625000
packet number:  417


In [ ]:
from fabric import ThreadingGroup
from invoke.exceptions import UnexpectedExit

# Define hosts with respective ports
hosts = [
    'PeterYao@pc605.emulab.net:30042',
    'PeterYao@pc603.emulab.net:30042',
    'PeterYao@pc604.emulab.net:30042',
    'PeterYao@pc760.emulab.net:22',
]

# Initialize the ThreadingGroup
group = ThreadingGroup(*hosts)

commands = [
    "wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip",
    "sudo apt update && sudo apt install -y unzip",
    "unzip l4s-testing.zip",
    "sudo dpkg --install debian_build/*",
    "sudo update-grub",
    "sudo reboot"
]

with group as g:
    for cmd in commands:
        try:
            print(f"Executing: {cmd}")
            g.run(cmd, hide=False)
        except UnexpectedExit as e:
            print(f"Command failed: {cmd}\nError: {e}")

In [ ]:
cmd = "hostname; uname -a"

with group as g:
    try:
        print(f"Executing: {cmd}")
        g.run(cmd, hide=False)
    except Exception as e:
        print(f"Command failed: {cmd}\nError: {e}")

In [ ]:
# setup the router queueing discipline
router_egress_name = "eno3"

cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=btl_capacity, buffer=btl_limit)
            
cmds_specific = "sudo tc qdisc add dev {iface} parent 1:3 handle 3: bfifo limit {buffer}".format(iface=router_egress_name, buffer=btl_limit)

router.run(cmds_prefix)    
router.run(cmds_specific)
router.run("sudo tc qdisc show dev {iface}".format(iface=router_egress_name))  




In [ ]:
router_egress_name = "eno3"

router.run("sudo tc qdisc show dev {iface}".format(iface=router_egress_name))  


In [ ]:
rx.sudo("ip netns exec ue1 sysctl -w net.ipv4.tcp_congestion_control=cubic")
rx.sudo("ip netns exec ue3 sysctl -w net.ipv4.tcp_congestion_control=cubic")

rx.sudo("ip netns exec ue1 sysctl -w net.ipv4.tcp_ecn=0")
rx.sudo("ip netns exec ue3 sysctl -w net.ipv4.tcp_ecn=0")

In [ ]:
rx.sudo("docker exec rfsim5g-oai-nr-ue1 sysctl net.ipv4.tcp_congestion_control")
rx.sudo("docker exec rfsim5g-oai-nr-ue1 sysctl -w net.ipv4.tcp_ecn=0")
rx.sudo("docker exec rfsim5g-oai-nr-ue1 sysctl net.ipv4.tcp_ecn")


rx.sudo("docker exec rfsim5g-oai-nr-ue2 sysctl net.ipv4.tcp_congestion_control")
rx.sudo("docker exec rfsim5g-oai-nr-ue2 sysctl -w net.ipv4.tcp_ecn=0")
rx.sudo("docker exec rfsim5g-oai-nr-ue2 sysctl net.ipv4.tcp_ecn")




In [ ]:
rx.run("ls -l")

In [ ]:
for c in conns:
    c.run("sudo sysctl -w net.ipv4.tcp_no_metrics_save=1")

In [ ]:
# rx.sudo("docker exec -it rfsim5g-oai-nr-ue1 apt-get update")
# rx.sudo("docker exec -it rfsim5g-oai-nr-ue1 apt-get install -y psmisc")
rx.sudo("docker exec -it rfsim5g-oai-nr-ue2 apt-get update", warn=True)
rx.sudo("docker exec -it rfsim5g-oai-nr-ue2 apt-get install -y psmisc", warn=True)

In [ ]:
# run the iperf command


rx.run("killall iperf3", warn=True)
rx.sudo("docker exec rfsim5g-oai-nr-ue1 killall iperf3", warn=True)
rx.sudo("docker exec rfsim5g-oai-nr-ue2 killall iperf3", warn=True)

router_egress_name = "eno3"
router.run("sudo ethtool -S {iface}".format(iface=router_egress_name))
router.run("ip -s link show {iface}".format(iface=router_egress_name))

rx.sudo("ip netns exec ue1 iperf3 -s -1 -p 4008 -D")
rx.sudo("ip netns exec ue3 iperf3 -s -1 -p 4008 -D")
# rx.sudo("docker exec rfsim5g-oai-nr-ue1 iperf3 -s -1 -p 4008 -D")
# rx.sudo("docker exec rfsim5g-oai-nr-ue2 iperf3 -s -1 -p 4008 -D")

local_file_path = r"d:\5g notes\5G-E2E-Wireless-Notes-OAI\exp-9-15\exp.sh"

# the monitor queue length shell script has already been copied to the router1 node
router.run("nohup ./monitor.sh eno3 60 1 > monitor.log 2>&1 &", pty=False)

# the put command is funny on windows, so I copy paster the exp file manually to the tx node
tx.run("chmod +x ~/exp.sh")
# monitor the RLC buffer. 
rx.run("/mydata/flexric/build/examples/xApp/c/monitor/xapp_gtp_mac_rlc_pdcp_moni > /dev/null 2>&1 &", pty=False)
tx.run("~/exp.sh cubic-ecn-none")



### download the database file with the biggest timestamp (the latest)

In [7]:
import re
import os

In [10]:
def download_latest_xapp_db(rx, remote_dir='/tmp/', pattern='xapp_db_*'):
    """
    Downloads the latest xapp_db_* file from the remote directory.

    Parameters:
    - rx: Fabric Connection object
    - remote_dir: Directory on the remote host to search for files
    - pattern: File pattern to match
    - local_dir: Local directory to download the file to
    """
    try:
        # Step 1: List all matching files
        list_cmd = f"ls {remote_dir}{pattern}"
        result = rx.run(list_cmd, hide=True, warn=True)

        if not result.ok:
            print("No files matching the pattern found.")
            return

        files = result.stdout.strip().split('\n')

        # Step 2: Filter out -shm and -wal files
        base_files = [f for f in files if not (f.endswith('-shm') or f.endswith('-wal'))]

        if not base_files:
            print("No base xapp_db_ files found (excluding -shm and -wal).")
            return

        # Step 3: Extract timestamps and find the latest file
        timestamp_pattern = re.compile(r'xapp_db_(\d+)$')
        latest_timestamp = -1
        latest_file = None

        for f in base_files:
            match = timestamp_pattern.search(f)
            if match:
                timestamp = int(match.group(1))
                if timestamp > latest_timestamp:
                    latest_timestamp = timestamp
                    latest_file = f
            else:
                print(f"File {f} does not match the expected pattern.")

        if latest_file is None:
            print("No valid timestamped xapp_db_ files found.")
            return

        print(f"Latest file found: {latest_file} with timestamp {latest_timestamp}")

        # Step 4: Download the latest file
        rx.get(latest_file)
        print(f"Downloaded {latest_file}.")
        return latest_file

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
# Call the function to download the latest xapp_db_* file
latest_file = download_latest_xapp_db(rx)


### Parse and Plot the database file

In [ ]:
import sqlite3
import pandas as pd

# Path to your SQLite file (replace 'your_database_file.sqlite' with your file)
db_file = 'xapp_db_1727534779300166'

# Connect to the SQLite database file
conn = sqlite3.connect(db_file)

# Fetch the list of tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)

table_name = 'RLC_bearer'
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Close the connection after extracting the data
conn.close()

# Display the DataFrame
print(df)

# # Save DataFrame to CSV if needed
df.to_csv(f'{db_file}.csv', index=False)


In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv('xapp_db_1727534779300166.csv')

# Display the first few rows of the DataFrame
print(df.head())

# Get the unique RNTI values (UE identifiers)
rntis = df['rnti'].unique()
print("Unique RNTI values:", rntis)

# Adjust 'tstamp' to start at zero and convert to seconds
# Since 'tstamp' is in microseconds, divide by 1,000,000 to get seconds
df['time_sec'] = (df['tstamp'] - df['tstamp'].min()) / 1_000_000

# Convert 'txbuf_occ_bytes' to Megabytes (MB)
df['txbuf_occ_MB'] = df['txbuf_occ_bytes'] / (1024 * 1024)

# Plot 'txbuf_occ_MB' over 'time_sec' for each UE
plt.figure(figsize=(12, 6))

for rnti in rntis:
    df_ue = df[df['rnti'] == rnti]
    plt.plot(df_ue['time_sec'], df_ue['txbuf_occ_MB'], label=f'UE {rnti}')

# Add a dashed horizontal line at 2 MB to represent the RLC buffer limit
plt.axhline(y=2, color='red', linestyle='--', label='RLC Buffer Limit (2 MB)')

# Add plot title and labels
plt.title('RLC Buffer Occupancy Over Time for Each UE')
plt.xlabel('Time (seconds)')
plt.ylabel('RLC Buffer Occupancy (MB)')

# Add legend and grid
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


### Plot the other Non-RLC information

In [ ]:
tx.close()
tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)
tx.get("cubic_ecn_none-result-ue1.json")
tx.get("cubic_ecn_none-result-ue2.json")
tx.get("cubic_ecn_none-ss-ue1.txt")
tx.get("cubic_ecn_none-ss-ue2.txt")

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Replace these with the paths to your actual iperf JSON result files
file1 = 'cubic_ecn_none-result-ue1.json'
file2 = 'cubic_ecn_none-result-ue2.json'

def load_iperf_data(filename):
    """Load iperf JSON data from a file and return a pandas DataFrame."""
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # Initialize lists to store the extracted data
    intervals = data['intervals']
    start_times = []
    end_times = []
    durations = []
    throughputs = []
    retransmissions = []
    
    # Iterate over each interval in the data
    for interval in intervals:
        sum_data = interval['sum']
        start_times.append(sum_data['start'])
        end_times.append(sum_data['end'])
        durations.append(sum_data['seconds'])
        # Convert throughput from bits per second to megabits per second
        throughput_mbps = sum_data['bits_per_second'] / 1_000_000  # Divide by 1,000,000
        throughputs.append(throughput_mbps)
        retransmissions.append(sum_data.get('retransmits', 0))  # Retransmits might not be present in UDP tests
    
    # Create a DataFrame
    df = pd.DataFrame({
        'Start Time': start_times,
        'End Time': end_times,
        'Duration': durations,
        'Throughput (Mbps)': throughputs,
        'Retransmissions': retransmissions
    })
    
    return df

def filter_zero_throughput(df):
    """Filter out data points where throughput is zero."""
    return df[df['Throughput (Mbps)'] != 0].reset_index(drop=True)

# Load data from both files
df1 = load_iperf_data(file1)
df2 = load_iperf_data(file2)

# Filter out zero throughput data points for plotting throughput
# df1_nonzero = filter_zero_throughput(df1)
# df2_nonzero = filter_zero_throughput(df2)

# Calculate overall throughput and average retransmissions for each dataset
overall_throughput1 = df1['Throughput (Mbps)'].mean()
average_retransmits1 = df1['Retransmissions'].mean()

overall_throughput2 = df2['Throughput (Mbps)'].mean()
average_retransmits2 = df2['Retransmissions'].mean()

print(f"UE 1 - Overall Throughput: {overall_throughput1:.2f} Mbps")
print(f"UE 1 - Average Retransmissions: {average_retransmits1:.2f}")

print(f"\nUE 2 - Overall Throughput: {overall_throughput2:.2f} Mbps")
print(f"UE 2 - Average Retransmissions: {average_retransmits2:.2f}")

# Plotting the throughputs over time (excluding zero throughput data points)
plt.figure(figsize=(12, 6))
plt.plot(df1['Start Time'], df1['Throughput (Mbps)'], label='UE 1 Throughput', marker='o')
plt.plot(df2['Start Time'], df2['Throughput (Mbps)'], label='UE 2 Throughput', marker='x')
plt.title('Throughput Over Time (Excluding Zero Values)')
plt.xlabel('Time (s)')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)
plt.show()

# Plotting the retransmissions over time (include all data points)
plt.figure(figsize=(12, 6))
plt.plot(df1['Start Time'], df1['Retransmissions'], label='UE 1 Retransmissions', marker='o')
plt.plot(df2['Start Time'], df2['Retransmissions'], label='UE 2 Retransmissions', marker='x')
plt.title('Retransmissions Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Number of Retransmissions')
plt.legend()
plt.grid(True)
plt.show()


COMMENT: the cwnd calculation is not showing the expected result. 

In [ ]:

name_tx0="cubic_ecn_none"


# the csv files generated is of the following format
# timestamp, fd, cwnd, srtt

file_out_tx0_csv = name_tx0+"-ss.csv"

for ue_id in range(1, 3):
    print("Running to generate csv files " + name_tx0)

    ss_tx0_script_processing="""

    f_1={types}; 
    ue_id={ue_id};
    rm -f ${{f_1}}-ss-${{ue_id}}.csv;
    cat ${{f_1}}-ss-${{ue_id}}.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT"> ${{f_1}}-ss-processed-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | awk '{{print $1}}' > ts-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\bcwnd:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' ' > cwnd-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\brtt:.*?(\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1   > srtt-${{f_1}}-${{ue_id}}.txt; 
    cat ${{f_1}}-ss-processed-${{ue_id}}.txt | grep -oP '\\bfd=.*?(\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-${{f_1}}-${{ue_id}}.txt;
    paste ts-${{f_1}}-${{ue_id}}.txt fd-${{f_1}}-${{ue_id}}.txt cwnd-${{f_1}}-${{ue_id}}.txt srtt-${{f_1}}-${{ue_id}}.txt -d ',' > ${{f_1}}-ss-${{ue_id}}.csv;""".format(types=name_tx0, ue_id="ue"+str(ue_id))

    tx.run(ss_tx0_script_processing)

tx.get("cubic_ecn_none"+"-ss-ue1.csv")
tx.get("cubic_ecn_none"+"-ss-ue2.csv")


In [18]:
import json
import pandas as pd

throughput_data = {}
srtt_data = {}

# Dictionaries to hold data for each UE
cwnd_data = {}
srtt_data_time = {}

for ue_id in range(1, 3):
    name_tx0 = "cubic_ecn_none"
    ue_str = "ue" + str(ue_id)

    # Load the JSON output file into a Python object
    with open(f"{name_tx0}-result-{ue_str}.json") as f:
        iperf3_data = json.load(f)

    throughput_data[name_tx0 + ue_str] = iperf3_data['end']['sum_received']['bits_per_second'] / (1000000 * 1)  # to convert Mbit

    # Average SRTT for Each Flow
    columns = ['timestamp', 'flow ID', 'cwnd', 'srtt']
    df_f1 = pd.read_csv(f"{name_tx0}-ss-{ue_str}.csv", names=columns)

    # Filter out rows with flow ID = 4, they are for the control flows
    df_f1 = df_f1[df_f1['flow ID'] != 4].reset_index(drop=True)

    average_RTT_f1 = df_f1['srtt'].mean()

    # Normalize the timestamps to start from zero
    df_f1['timestamp'] = df_f1['timestamp'] - df_f1['timestamp'].iloc[0]

    # Save cwnd and srtt data into separate dataframes for each UE
    cwnd_data[ue_str] = df_f1[['timestamp', 'cwnd']]
    srtt_data_time[ue_str] = df_f1[['timestamp', 'srtt']]

    srtt_data[name_tx0 + ue_str] = average_RTT_f1

# Save throughput_data to a JSON file
with open('throughput_data.json', 'w') as f:
    json.dump(throughput_data, f)

# Save srtt_data to a JSON file
with open('srtt_data.json', 'w') as f:
    json.dump(srtt_data, f)

# Save cwnd_data and srtt_data_time to CSV files for each UE
for ue_str in cwnd_data.keys():
    cwnd_data[ue_str].to_csv(f"cwnd_data_{ue_str}.csv", index=False)
    srtt_data_time[ue_str].to_csv(f"srtt_data_time_{ue_str}.csv", index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize lists to hold data
timestamps = []
cwnds = []
ue_ids = []

# Loop through UEs to read their data
for ue_id in range(1, 3):
    ue_str = "ue" + str(ue_id)
    filename = f'cwnd_data_{ue_str}.csv'
    
    # Read the CSV file for the UE
    df = pd.read_csv(filename)
    
    # Append data to lists
    timestamps.append(df['timestamp'])
    cwnds.append(df['cwnd'] * 1448)  # Convert cwnd to bytes
    ue_ids.append(ue_str)

# Create a figure with two subplots side by side
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Plot for each UE
for i, ue_str in enumerate(ue_ids):
    axes[i].plot(timestamps[i], cwnds[i], marker='', label=ue_str)
    axes[i].set_title(f'Congestion Window (cwnd) Over Time - {ue_str.upper()}')
    axes[i].set_xlabel('Time (seconds)')
    axes[i].set_ylabel('cwnd (bytes)')
    axes[i].grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
router.get("monitor.log")

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt

# Replace with your actual filename
filename = 'monitor.log'

def parse_qdisc_data(filename):
    """Parse qdisc data from a file and return a DataFrame."""
    timestamps = []
    sent_bytes = []
    sent_packets = []
    dropped_packets = []
    overlimits = []
    backlog_bytes = []
    backlog_packets = []

    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            # Extract timestamp
            match_time = re.match(r'^(\d+\.\d+)', line)
            if match_time:
                timestamp = float(match_time.group(1))
                timestamps.append(timestamp)
            else:
                continue  # Skip line if timestamp not found

            # Extract bfifo qdisc statistics
            # Assuming 'qdisc bfifo' appears after 'qdisc htb' in the line
            bfifo_data = line.split('qdisc bfifo')[1]

            # Extract Sent bytes and packets
            match_sent = re.search(r'Sent\s+(\d+)\s+bytes\s+(\d+)\s+pkt', bfifo_data)
            if match_sent:
                sent_bytes.append(int(match_sent.group(1)))
                sent_packets.append(int(match_sent.group(2)))
            else:
                sent_bytes.append(None)
                sent_packets.append(None)

            # Extract dropped packets
            match_dropped = re.search(r'dropped\s+(\d+)', bfifo_data)
            if match_dropped:
                dropped_packets.append(int(match_dropped.group(1)))
            else:
                dropped_packets.append(None)

            # Extract overlimits
            match_overlimits = re.search(r'overlimits\s+(\d+)', bfifo_data)
            if match_overlimits:
                overlimits.append(int(match_overlimits.group(1)))
            else:
                overlimits.append(None)

            # Extract backlog bytes and packets
            match_backlog = re.search(r'backlog\s+(\d+)b\s+(\d+)p', bfifo_data)
            if match_backlog:
                backlog_bytes.append(int(match_backlog.group(1)))
                backlog_packets.append(int(match_backlog.group(2)))
            else:
                backlog_bytes.append(None)
                backlog_packets.append(None)

    # Create a DataFrame
    df = pd.DataFrame({
        'Timestamp': timestamps,
        'Sent Bytes': sent_bytes,
        'Sent Packets': sent_packets,
        'Dropped Packets': dropped_packets,
        'Overlimits': overlimits,
        'Backlog Bytes': backlog_bytes,
        'Backlog Packets': backlog_packets
    })

    # Convert timestamps to relative time (seconds since start)
    df['Relative Time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    return df

def plot_queue_length(df):
    """Plot queue length (backlog) over time."""
    plt.figure(figsize=(12, 6))
    plt.plot(df['Relative Time'], df['Backlog Bytes'], marker='o')
    plt.title('Queue Length (Backlog) Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Backlog (bytes)')
    plt.grid(True)
    plt.show()

def plot_packet_drop_rate(df):
    """Plot packet drop rate over time."""
    # Calculate the difference in dropped packets between measurements
    df['Dropped Packets Diff'] = df['Dropped Packets'].diff().fillna(0)
    
    print(df['Dropped Packets Diff'])

    plt.figure(figsize=(12, 6))
    plt.plot(df['Relative Time'], df['Dropped Packets Diff'], marker='x', color='red')
    plt.title('Packet Drop Rate Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Dropped Packets per Interval')
    plt.grid(True)
    plt.show()

def analyze_backlog(df):
    """Analyze backlog and determine if there is a backlog."""
    max_backlog = df['Backlog Bytes'].max()
    if max_backlog > 0:
        print(f"There is a backlog. Maximum backlog is {max_backlog} bytes.")
    else:
        print("There is no backlog. The queue is empty throughout the measurements.")

# Main execution
df = parse_qdisc_data(filename)

# Plot queue length
plot_queue_length(df)

# Plot packet drop rate
plot_packet_drop_rate(df)

# Analyze backlog
analyze_backlog(df)


In [ ]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Read the SRTT data for each UE
ue1_file_path = 'srtt_data_time_ue1.csv'  # Replace with your actual filename for UE1
ue2_file_path = 'srtt_data_time_ue2.csv'  # Replace with your actual filename for UE2

# Reading the CSV files
df_ue1 = pd.read_csv(ue1_file_path)
df_ue2 = pd.read_csv(ue2_file_path)

# Normalize timestamps to start from zero for better comparison
df_ue1['timestamp'] = df_ue1['timestamp'] - df_ue1['timestamp'].iloc[0]
df_ue2['timestamp'] = df_ue2['timestamp'] - df_ue2['timestamp'].iloc[0]

# Calculate the average SRTT for each UE
avg_ue1 = df_ue1['srtt'].mean()
avg_ue2 = df_ue2['srtt'].mean()

# Plot the SRTT changes over time for both UE1 and UE2
plt.figure(figsize=(12, 6))
plt.plot(df_ue1['timestamp'], df_ue1['srtt'], label='UE1 SRTT', marker='o')
plt.plot(df_ue2['timestamp'], df_ue2['srtt'], label='UE2 SRTT', marker='s')

# Plot average lines for each UE
plt.axhline(y=avg_ue1, color='blue', linestyle='--', linewidth=1, label=f'Average UE1 ({avg_ue1:.2f} ms)')
plt.axhline(y=avg_ue2, color='orange', linestyle='--', linewidth=1, label=f'Average UE2 ({avg_ue2:.2f} ms)')

# Adding labels and title
plt.xlabel('Time (seconds)')
plt.ylabel('SRTT (ms)')
plt.title('SRTT Changes Over Time for UE1 and UE2')

# Adding a legend and grid
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Display the plot
plt.tight_layout()
plt.show()

# Print the average SRTT for each UE
print(f"Average SRTT for UE1: {avg_ue1:.3f} ms")
print(f"Average SRTT for UE2: {avg_ue2:.3f} ms")
